# Examine genetic clines

In [1]:
import os
import allel
import numpy as np
import pandas as pd
import vcf

from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches

from pathlib import Path
from haversine import haversine
from skbio.stats.distance import mantel

import cartopy.crs as ccrs
import cartopy.feature as cf
from pykrige.ok import OrdinaryKriging

from intervaltree import Interval, IntervalTree


In [2]:
proj_dir="/master/nplatt/sch_hae_scan"
results_dir="{}/results".format(proj_dir)

os.chdir(proj_dir)

In [3]:
Path("{}/clines".format(results_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/clines".format(results_dir))

In [4]:
#read in sample info
info_df=pd.read_csv("{}/data/seq_and_sample_docs/all_sh_sb_sample_data.csv".format(proj_dir), sep=",") 

#get the pca kmeans groups
pca_df=pd.read_csv("{}/results/pca/pca_df.csv".format(proj_dir), sep=",")
pca_df=pca_df[["sample_name", "kmeans_group", "kmeans_label"]]

#add to the sample info
info_df=info_df.merge(pca_df, how='right', on='sample_name')

In [5]:
Path("{}/masked_clines".format(results_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/masked_clines".format(results_dir))

# mask sb alleles (w/VCFTOOLS)

## Read in the rfmix data and get an autosomal vcf file of only the query SH samples from rfmix run

In [6]:
#read in rfmix table
rf_df=pd.read_csv(f"{results_dir}/rfmix/rfmix_df.csv")
rf_df

,chrom,s_pos,e_pos,s_gpos,e_gpos,n_snps,Sb_NG_au_1.2.0,Sb_NG_au_1.2.1,Sb_NG_au_2.13.0,Sb_NG_au_2.13.1,...,ssp_niger_libore_166.0,ssp_niger_libore_166.1,ssp_niger_libore_167.0,ssp_niger_libore_167.1,ssp_niger_libore_168.0,ssp_niger_libore_168.1,ssp_niger_libore_169.0,ssp_niger_libore_169.1,ssp_zambia_kafue_71.0,ssp_zambia_kafue_71.1
0,NC_067196.1,124225,212192,0.43,0.74,1302,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
1,NC_067196.1,212192,289797,0.74,1.01,860,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,NC_067196.1,289797,302958,1.01,1.06,645,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
3,NC_067196.1,302958,623463,1.06,2.17,2980,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,NC_067196.1,623463,625997,2.17,2.18,25,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39472,NC_067199.1,46758459,46761155,162.92,162.93,365,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
39473,NC_067199.1,46761155,46761224,162.93,162.93,15,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
39474,NC_067199.1,46761224,46762583,162.93,162.94,115,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
39475,NC_067199.1,46762583,46763230,162.94,162.94,75,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1


In [7]:
rf_samples = rf_df.columns[6:]
rf_samples = list(set([s[:-2] if s.endswith('.0') or s.endswith('.1') else s for s in rf_samples]))

In [8]:
len(rf_samples)

111

In [9]:
if os.path.exists("sh_se.samples.list"):
    os.remove("sh_se.samples.list")
    
if os.path.exists("sh_nw.samples.list"):
    os.remove("sh_nw.samples.list")


for sample in rf_samples:
    try:
        pop = info_df.dropna(axis=0, subset=["lat", "lon"]).loc[info_df["sample_name"] == sample, "kmeans_label" ].values[0]
        with open(f"{pop}.samples.list", 'a') as f:
            f.write(f"{sample}\n")
            
    except:
        next

In [10]:
%%bash

cat sh_se.samples.list sh_nw.samples.list >sh.samples.list
wc -l sh.samples.list


89 sh.samples.list


In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    vcftools \
        --vcf  ~/sch_hae_scan/results/filter_genotypes/sorted_annotated_snps.vcf \
        --chr NC_067196.1 \
        --chr NC_067197.1 \
        --chr NC_067198.1 \
        --chr NC_067199.1 \
        --chr NC_067200.1 \
        --chr NC_067201.1 \
        --chr NC_067202.1 \
        --keep sh.samples.list \
        --recode \
        --recode-INFO-all \
        --out sh_rf_autosomes

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream bgzip sh_rf_autosomes.recode.vcf
conda run -n scan-clines --cwd . --live-stream tabix -p vcf sh_rf_autosomes.recode.vcf.gz

In [11]:
%%bash

cat sh.samples.list | parallel -j 48 "conda run -n scan-clines --cwd . --live-stream bcftools view -s {} sh_rf_autosomes.recode.vcf.gz -o {}.vcf"

## Now find all the locations of Sb alleles in each sample and create a bed file for masking

In [12]:
se_samples = np.loadtxt("sh_se.samples.list", dtype=str)
nw_samples = np.loadtxt("sh_nw.samples.list", dtype=str)

sh_samples = np.concatenate((se_samples, nw_samples))

sh_haplotypes = [sample + suffix for sample in sh_samples for suffix in [".0", ".1"]]


#intersection_cols = list(set(rf_df.columns) & set(sh_haplotypes))

# Combine the individual strings and the list from the intersection
cols_to_use = ["chrom", "s_pos", "e_pos", "n_snps"] + sh_haplotypes


In [ ]:
to_mask_df=rf_df[cols_to_use]
to_mask_df["samples"]=None
to_mask_df

In [ ]:
mask_samples = list(to_mask_df.columns[4:-1])
mask_samples = [s[:-2] if s.endswith('.1') or s.endswith('.0') else s for s in mask_samples]
mask_samples = list(set(mask_samples))
mask_samples

In [ ]:
len(mask_samples)

In [ ]:
sb_allele = 0

for sample in mask_samples:
    sample_to_mask_bed = to_mask_df.loc[((to_mask_df[f"{sample}.0"] == sb_allele) | (to_mask_df[f"{sample}.1"] == sb_allele)), ["chrom", "s_pos", "e_pos"]]
    sample_to_mask_bed["s_pos"] = sample_to_mask_bed["s_pos"]-1
    sample_to_mask_bed["e_pos"] = sample_to_mask_bed["e_pos"]+1
    sample_to_mask_bed.to_csv(f"{sample}.mask.bed", sep="\t", header=False, index=False)

## Mask each sample and then merge into a single masked.vcf

In [ ]:
%%bash
cat sh.samples.list | parallel -j 48 "conda run -n scan-clines --cwd . --live-stream vcftools --vcf {}.vcf --exclude-bed {}.mask.bed --recode --recode-INFO-all --out {}.masked"

In [ ]:
cat sh.samples.list | parallel -j 48 "conda run -n scan-clines --cwd . --live-stream bgzip {}.masked.recode.vcf"
cat sh.samples.list | parallel -j 48 "conda run -n scan-clines --cwd . --live-stream tabix -p vcf {}.masked.recode.vcf.gz"
ls *.masked.recode.vcf.gz >masked.list

In [ ]:
#https://samtools.github.io/bcftools/bcftools.html#merge
bcftools --merge snps --file-list masked.list --output masked.vcf --output-type v

## Filter and prep the masked vcf for downstream analyses

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    vcftools --missing-site --vcf masked.vcf

In [ ]:
miss_df=pd.read_csv("out.lmiss", sep="\t", header=0)

In [ ]:
plt.hist(miss_df["F_MISS"], bins=25)
plt.show()

In [ ]:
(miss_df["F_MISS"]<0.1).sum()

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    vcftools \
    --max-missing 0.9 \
    --vcf masked.vcf \
    --recode \
    --recode-INFO-all \
    --out max_miss_masked

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    plink \
        --vcf max_miss_masked.recode.vcf \
        --allow-extra-chr \
        --double-id \
        --indep-pairwise 25 5 0.20 \
        --out max_miss_masked_ld

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    vcftools \
        --vcf max_miss_masked.recode.vcf \
        --exclude max_miss_masked_ld.prune.out \
        --recode \
        --recode-INFO-all \
        --out max_miss_masked_ld

In [ ]:
%%bash

conda run -n scan-clines --cwd . --live-stream \
    vcftools \
        --vcf max_miss_masked_ld.recode.vcf \
        --thin 10000 \
        --recode \
        --recode-INFO-all \
        --out minus_sb_mantel

## Calculate genetic distances 
calculated wtih `VCF2Dis` from https://github.com/BGI-shenzhen/VCF2Dis.  Cloned into `sch_hae_scan/bin`

In [ ]:
#get the genetic data
vcf_reader = vcf.Reader(open('minus_sb_mantel.recode.vcf', 'r'))
#get the sample order
samples=vcf_reader.samples

In [ ]:
%%bash

~/sch_hae_scan/bin/VCF2Dis/bin/VCF2Dis -InPut minus_sb_mantel.recode.vcf -OutPut minus_sb_mantel_p_distance.tsv

In [ ]:
#read in distance matrix
gen_df = pd.read_table("minus_sb_mantel_p_distance.tsv", sep="\t", header=None, index_col=0, skiprows=[0])

#VCF2Dis only keeps the first 20 chars in the sample name... so need to re-header
gen_df.index=samples
gen_df.columns=samples
gen_df.to_csv("minus_sb_mantel_p_distance.csv", sep=",")
gen_df

## Get geo distances

In [ ]:
km_dists=np.array([])
for s1 in samples:
    s1_lat=info_df.loc[info_df["sample_name"] == s1]["lat"].values[0]
    s1_lon=info_df.loc[info_df["sample_name"] == s1]["lon"].values[0]
    
    for s2 in samples:
        s2_lat=info_df.loc[info_df["sample_name"] == s2]["lat"].values[0]
        s2_lon=info_df.loc[info_df["sample_name"] == s2]["lon"].values[0]
        
        km_dists=np.append(km_dists, haversine((s1_lat, s1_lon), (s2_lat, s2_lon)))
        
#reshape into a 2d matrix
km_dists.shape=(len(samples), len(samples))

#convert to a df
km_df=pd.DataFrame(data=km_dists, columns=samples, index=samples)
km_df.to_csv("mantel_geo_distance.csv", sep=",")
km_df

## Mantel Tests

In [ ]:
# Need to do 4 comparisons
# Sb
# Sh
# NW
# SE

#get idecies of each sample type/species of interest

labels = []

for sample in samples:
    labels.append(pca_df.loc[pca_df["sample_name"] == sample, "kmeans_label"].values[0])
    
sb_idx=matching_indices = [i for i, s in enumerate(labels) if s == "sb"]
nw_idx=matching_indices = [i for i, s in enumerate(labels) if s == "sh_nw"]
se_idx=matching_indices = [i for i, s in enumerate(labels) if s == "sh_se"]
sh_idx=nw_idx + se_idx


In [ ]:
gen_df

In [ ]:
for idx, color, label in [(se_idx, "green", "sh_se"),
                          (nw_idx, "red", "sh_nw") ]:
    
    #subsample genetic and physical distance matricies
    g=gen_df.iloc[idx, idx]
    k=km_df.iloc[idx, idx]

    #conduct the mantel test
    r2, p, n = mantel(g, k, permutations=1000)
    print("Mantel test - {}: r2={:.2f}, p={:.3f}, n={}".format(label, r2, p, n))

## Plot d-distance vs geo-distance

In [ ]:
samples_with_geo_data = info_df.iloc[info_df['lat'].notnull().values]["sample_name"].values

for idx, color, label in [(se_idx, "green", "sh_se"),
                          (nw_idx, "red", "sh_nw") ]:
    
    #subsample genetic and physical distance matricies
    g=gen_df.iloc[idx, idx]
    k=km_df.iloc[idx, idx]

    #remove duplicate values for plotting
    mask_upper = np.triu(np.ones(g.shape)).astype(bool)
    
    g.where(~mask_upper, np.nan, inplace=True)
    k.where(~mask_upper, np.nan, inplace=True)
   
    gs=g.values.flatten()
    ks=k.values.flatten()

    gs=gs[~np.isnan(gs)]
    ks=ks[~np.isnan(ks)]

    #plot scatter plot
    plt.scatter(ks, gs, alpha=0.5, color=color, edgecolors="black", linewidth=0.3, label=label)

    
plt.legend()
plt.title("Isolation by distance")
plt.xlabel("Kilometers")
plt.ylabel("P-distance")

# Display the figure
plt.savefig("minus_sb_p-dist_vs_geo-dist.png", dpi=600, bbox_inches='tight')
plt.savefig("minus_sb_p-dist_vs_geo-dist.svg", bbox_inches='tight')


plt.show()

## Plot p-distance to a ref sample vs. lat and long

In [ ]:
sh_samples = np.concatenate((sh_se_samples, sh_nw_samples))
countries=info_df.loc[info_df["sample_name"].isin(sh_samples), "country"].unique()
colors = cm.rainbow(np.linspace(0, 1, len(countries)))
country_colors = dict(zip(countries, colors))

In [ ]:
g = gen_df.iloc[sh_idx, sh_idx]
k = km_df.iloc[sh_idx, sh_idx]

ref_sample = "sha_unguja_kinyasini_19"
#ref_sample=info_df.iloc[info_df["lat"].astype(float).idxmin()]["sample_name"]

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Scatter plot for Latitude
for sample, p_distance in g[ref_sample].items():
    if sample != ref_sample:
        country = info_df.loc[info_df["sample_name"] == sample, "country"].item()
        latitude = info_df.loc[info_df["sample_name"] == sample, "lat"].item()
        ax1.scatter(latitude, p_distance, color=country_colors[country], edgecolors="black", linewidth=0.3)
ax1.set_xlabel("Latitude")
ax1.set_ylabel("P-distance")

# Scatter plot for Longitude
for sample, p_distance in g[ref_sample].items():
    if sample != ref_sample:
        longitude = info_df.loc[info_df["sample_name"] == sample, "lon"].item()
        country = info_df.loc[info_df["sample_name"] == sample, "country"].item()
        ax2.scatter(longitude, p_distance, color=country_colors[country], edgecolors="black", linewidth=0.3)
ax2.set_xlabel("Longitude")
ax2.set_ylabel("P-distance")

# Create a custom legend for the entire figure
legend_handles = [mpatches.Patch(color=country_colors[country], label=country) for country in countries]

# Positioning the legend can be done using bbox_to_anchor. 
# Here, I'm placing it outside the second subplot to the right.
fig.legend(handles=legend_handles, loc='center right', bbox_to_anchor=(1.2, 0.5))


# Display the figure
plt.savefig("minus_sb_p-distance_lat_lon_gradient.png", dpi=600, bbox_inches='tight')
plt.savefig("minus_sb_p-distance_lat_lon_gradient.svg", bbox_inches='tight')


plt.show()
